# Imports

In [443]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


pd.set_option('display.max_columns', None)

# Datensätze

In [444]:
df_stores = pd.read_csv("dmml1_stores.csv")
df_test = pd.read_csv("dmml1_test.csv")
df_train = pd.read_csv("dmml1_train.csv")

In [445]:
df_stores

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
1,2,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
2,3,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
3,4,a,c,1070.0,NaN,NaN,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
4,5,a,c,4590.0,3.0,2000.0,1,40.0,2011.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...
295,296,a,a,690.0,6.0,2007.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
296,297,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
297,298,d,c,1340.0,10.0,2006.0,1,5.0,2013.0,"Feb,May,Aug,Nov"
298,299,d,a,260.0,2.0,2012.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [446]:
df_train

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,4,2015-04-30,6884,716,1,1,0,0
1,1,3,2015-04-29,6764,756,1,1,0,0
2,1,2,2015-04-28,6861,678,1,1,0,0
3,1,1,2015-04-27,6523,647,1,1,0,0
4,1,7,2015-04-26,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0
246899,300,5,2013-01-04,5524,646,1,0,0,1
246900,300,4,2013-01-03,5563,718,1,0,0,1
246901,300,3,2013-01-02,6218,753,1,0,0,1


In [447]:
df_test

,Store ID,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,1,1,0,1
1,1,4,2015-07-30,1,1,0,1
2,1,3,2015-07-29,1,1,0,1
3,1,2,2015-07-28,1,1,0,1
4,1,1,2015-07-27,1,1,0,1
...,...,...,...,...,...,...,...
27595,300,2,2015-05-05,1,1,0,0
27596,300,1,2015-05-04,1,1,0,0
27597,300,7,2015-05-03,0,0,0,0
27598,300,6,2015-05-02,1,0,0,0


<br>

# Aufgabe 1: 
1. Wie viele Datenpunkte gibt es pro Store?
2. Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und beschreiben Sie typische Muster, die Sie erkennen
3. Beschreiben Sie einige prägnante Zusammenhänge, die Ihnen zwischen verschiedenen Features auffallen

<br>

## Datenpunkte pro Store

In [448]:
df_group_train = df_train.groupby("Store ID").size().reset_index(name= "Datenpunkte pro Store")
df_group_train

,Store ID,Datenpunkte pro Store
0,1,850
1,2,850
2,3,850
3,4,850
4,5,850
...,...,...
295,296,850
296,297,850
297,298,850
298,299,666


In [449]:
df_group_train["Datenpunkte pro Store"].unique()

array([850, 666, 849], dtype=int64)

## Visualisierung der Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und Erfassung typischer Muster

In [450]:
random_store_ids = np.random.choice(df_train['Store ID'].unique(), size=5, replace=False)
random_store_ids

df_train_filtered = df_train[df_train["Store ID"].isin(random_store_ids)]

fig = px.histogram(
    df_train_filtered,
    x='Sales',
    color='Store ID',
    title='Verteilung der Verkaufszahlen für ausgewählte Stores pro Tag',
    labels={'Sales': 'Verkaufszahlen'}
)

fig.update_layout(bargap=0.1)  
fig.show()

Man kann erkennen, dass alle Stores fast genau so oft keinen Umsatz an einem Tag machen, was an den Öffnungstagen des jeweiligen Stores, die sich sehr ähneln, liegen wird.
Beim Rest kann man erkennen, dass dieser eine rechtsschiefe Verteilung aufweist, bei der viele Stores oft mittelohe, nur vereinzelte Stores nicht sehr oft wenig hohe und einige Stores selten sehr hohe Umsätze erzielen.

<br>

## Auffälligkeiten zwischen prägnanten Features

In [451]:
#Zusammenführen von df_stores und df_train
df_merge_train_stores = pd.merge(df_stores, df_train, how = "outer", on = "Store ID")
df_merge_train_stores

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4,2015-04-30,6884,716,1,1,0,0
1,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",3,2015-04-29,6764,756,1,1,0,0
2,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2,2015-04-28,6861,678,1,1,0,0
3,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",1,2015-04-27,6523,647,1,1,0,0
4,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",7,2015-04-26,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,6,2013-01-05,5194,569,1,0,0,0
246899,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,5,2013-01-04,5524,646,1,0,0,1
246900,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,4,2013-01-03,5563,718,1,0,0,1
246901,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,3,2013-01-02,6218,753,1,0,0,1


### DayOfWeek und Sales

In [452]:
#Summe aller Umsätze pro Wochentag
df = df_merge_train_stores.groupby("DayOfWeek")["Sales"].sum().reset_index()
df

,DayOfWeek,Sales
0,1,277782407
1,2,249158860
2,3,234329114
3,4,225847531
4,5,240166697
5,6,211325985
6,7,4411988


In [453]:
correlation = np.corrcoef(df_merge_train_stores['DayOfWeek'], df_merge_train_stores['Sales'])[0,1]
correlation


-0.4479479398594474

Der Umsatz ist montags am höchsten und sonntags am niedrigsten, was sich auch in der berechneten Korrelation wiederspiegelt.

<br>

### StateHoliday, SchoolHoliday und Sales

In [454]:
#Umsatz in Abhängigkeit von StateHoliday und SchoolHoliday
df_group_state_hol = df_train.groupby(["StateHoliday", "SchoolHoliday"]).agg({"Sales": "sum", "DayOfWeek": "size"}).reset_index()
df_group_state_hol.columns = ["StateHoliday", "SchoolHoliday", "Gesamtumsatz", "Anzahl_Tage"]
df_group_state_hol["Durchschnittlicher_Umsatz_pro_Tag"] = df_group_state_hol["Gesamtumsatz"] / df_group_state_hol["Anzahl_Tage"]

df_group_state_hol


,StateHoliday,SchoolHoliday,Gesamtumsatz,Anzahl_Tage,Durchschnittlicher_Umsatz_pro_Tag
0,0,0,1155876892,199634,5789.980124
1,0,1,285936257,40009,7146.798395
2,a,0,756170,3024,250.056217
3,a,1,117144,1324,88.477341
4,b,0,0,108,0.000000
5,b,1,239326,1692,141.445626
6,c,1,96793,1112,87.044065


In [455]:
correlation = np.corrcoef(df_merge_train_stores['SchoolHoliday'],  df_merge_train_stores['Sales'])[0,1]
correlation


0.07500254197292291

Der durchschnittliche Umsatz pro Tag ist am höchsten, wenn kein Feiertag ist und gleichzeitig Schulferien sind

<br>

### Promo, Promo2 und Sales

In [456]:
df = df_merge_train_stores.groupby(["Promo", "Promo2"])["Sales"].mean().reset_index(name="Durchschnittlicher_Umsatz")
df

,Promo,Promo2,Durchschnittlicher_Umsatz
0,0,0,4927.276707
1,0,1,3955.624107
2,1,0,8691.654629
3,1,1,7335.541349


In [457]:
correlation_matrix = np.corrcoef(df_merge_train_stores[['Promo', 'Promo2', 'Sales']], rowvar=False)
promo_sales_correlation = correlation_matrix[0, 2]
promo2_sales_correlation = correlation_matrix[1, 2]

promo2_sales_correlation

-0.13942459332083526

In [458]:
promo_sales_correlation

0.43476715921066506

Eine Promotion an dem jeweiligen Tag scheint den Umsatz moderat zu verbessern, während eine eine fortlaufende Werbeaktion dies auf den ersten Blick erstmal nicht bewirkt.

<br>

# Aufgabe 2:
Führen Sie geeignete Vorverarbeitungsschritte durch, z.B. Berhandlung von Ausreißern und fehlenden Werten, Skalierung der Features


# Aufgabe 3:
Generieren Sie neue Features, die für die Vorhersage des Umsatzes aussagekräftig sein können

In [459]:
#Umwandlung von "Date" in Datumsformat
df_merge_train_stores['Date'] = pd.to_datetime(df_merge_train_stores['Date'])

#Auffüllen der NaN-Werte von "PromoIntervall"
df_merge_train_stores["PromoInterval"] = df_merge_train_stores["PromoInterval"].fillna("NoPromoInterval")

#Erstellung neuer Datumsfeatures
df_merge_train_stores['Tag'] = df_merge_train_stores['Date'].dt.day
df_merge_train_stores['Monat'] = df_merge_train_stores['Date'].dt.month
df_merge_train_stores['Jahr'] = df_merge_train_stores['Date'].dt.year

#Feature, das anzeigt ob Wochenende ist
df_merge_train_stores['IsWeekend'] = df_merge_train_stores['Date'].dt.dayofweek.isin([6, 7]).astype(int)

#Feature, das die Tage bis Weihnachten zählt
df_merge_train_stores['DaysUntilChristmas'] = (pd.to_datetime(df_merge_train_stores['Date'].dt.year.astype(str) + '-12-25') - df_merge_train_stores['Date']).dt.days
df_merge_train_stores.loc[df_merge_train_stores['DaysUntilChristmas'] < 0, 'DaysUntilChristmas'] = (pd.to_datetime((df_merge_train_stores['Date'].dt.year + 1).astype(str) + '-12-25') - df_merge_train_stores['Date']).dt.days

In [460]:
#Funktion, um Ausreißer zu entfernen
def remove_outliers(df):
    sales_col = 'Sales'
    mask = (df[sales_col] >= np.percentile(df[sales_col], 1)) & (df[sales_col] <= np.percentile(df[sales_col], 99))
    df = df[mask]
    return df


### Column Transformer mit Promo2 (unskaliert)

In [461]:

#Tabelle Speichern, die keine Ausreißer mehr enthält
df_merge_train_stores = remove_outliers(df_merge_train_stores)             


column_trans = ColumnTransformer([("One-hot-enc", OneHotEncoder(handle_unknown="ignore"), ['StateHoliday', 'StoreType','Assortment', "PromoInterval"]),                                
                                  ("NaN_to_mean", make_pipeline(SimpleImputer(strategy="mean"), StandardScaler()),["CompetitionDistance", 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']),
                                  ("fill_zeros", SimpleImputer(strategy='constant', fill_value=0), ["Promo2SinceWeek", "Promo2SinceYear"] ),
                                  ("Entfernen von Spalten", "drop", ["Date"])
                                  ], remainder="passthrough")


<br>

# Aufgabe 4:
Trainieren Sie drei verschiedene Modelle, die in der Vorlesung behandelt wurden: ein lineares Modell (einfache lineare Regression, Ridge, Lasso), einen Entscheidungsbaum und ein Ensemble-Modell (Gradient Boosting oder Random Forest)

In [462]:
#Features und Label aus zusammmenführtem Traingsdatensatz herausfiltern 

X_train = df_merge_train_stores.drop(columns=["Sales", "Customers"]) #Date wird in column_trans gedroppt
y_train = df_merge_train_stores["Sales"]

### Hyperparameteroptimierung mit Kreuzvalidierung

In [463]:
def kreuzvalidierung_mit_hyperparameter(model_type, alpha_set=[0.01, 0.1, 0.5, 1, 2], max_depth_set=[2,5,10,None], min_samples_split_set=[2,5]):

    alpha_set = alpha_set
    max_depth_set = max_depth_set
    min_samples_split_set = min_samples_split_set

    X_train_fold, X_test_fold, y_train_fold, y_test_fold = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    kf = KFold(n_splits=5)

    if model_type.lower() == "ridge" or model_type.lower() == "lasso":        #Falls ein lineares Modell vorliegt

        df_R_2 = pd.DataFrame(columns=["r2_train", "r2_test", "alpha", "validation_dataset"])

        for alpha in alpha_set:
            for i, (train_index, test_index) in enumerate(kf.split(X_train)):

                if model_type.lower() == "ridge":
                        pipe = Pipeline([("Vorverarbeitung", column_trans), ("ridge regression", linear_model.Ridge(alpha=alpha))])
                elif model_type.lower() == "lasso":
                        pipe = Pipeline([("Vorverarbeitung", column_trans), ("lasso regression", linear_model.Lasso(alpha=alpha))])

                pipe.fit(X_train_fold, y_train_fold)

                y_train_pred = pipe.predict(X_train_fold)
                y_test_pred = pipe.predict(X_test_fold)

                r2_train = r2_score(y_train_fold, y_train_pred)
                r2_test = r2_score(y_test_fold, y_test_pred)

                r2_train = pipe.score(X_train_fold, y_train_fold)
                r2_test = pipe.score(X_test_fold, y_test_fold)

                df_R_2 = df_R_2.append({"r2_train": r2_train, 'r2_test': r2_test, 'alpha': alpha, "validation_dataset": i+1}, ignore_index=True)


    elif (model_type.lower() == "tree") or (model_type.lower() == "random_forest"):   #Falls ein Baummodell vorliegt

        df_R_2 = pd.DataFrame(columns=["r2_train", "r2_test", "max_depth", "min_samples_split", "validation_dataset"])

        for max_depth in max_depth_set:
            for min_samples_split in min_samples_split_set:
                for i, (train_index, test_index) in enumerate(kf.split(X_train)):

                    if model_type.lower() == "tree":
                        pipe = Pipeline([("Vorverarbeitung", column_trans), ("tree regression", DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split))])
                    elif model_type.lower() == "random_forest":
                        pipe = Pipeline([("Vorverarbeitung", column_trans), ("random_forest regression", RandomForestRegressor(max_depth=max_depth, min_samples_split=min_samples_split))])

                    pipe.fit(X_train_fold, y_train_fold)

                    y_train_pred = pipe.predict(X_train_fold)
                    y_test_pred = pipe.predict(X_test_fold)

                    r2_train = r2_score(y_train_fold, y_train_pred)
                    r2_test = r2_score(y_test_fold, y_test_pred)

                    r2_train = pipe.score(X_train_fold, y_train_fold)
                    r2_test = pipe.score(X_test_fold, y_test_fold)

                    df_R_2 = df_R_2.append({"r2_train": r2_train, 'r2_test': r2_test,
                                            'max_depth': max_depth, 'min_samples_split': min_samples_split,
                                            "validation_dataset": i+1}, ignore_index=True)


    print("Vergleichstabelle der R^2 Werte:\n")
    print(df_R_2.to_markdown(index=False))

    print("\n Folgende Kombination aus Hyperparameter und Validierungsdatensatz ist die beste: \n")
    print(df_R_2.loc[df_R_2['r2_test'] == df_R_2['r2_test'].max()])


<br>

## Lineares Modell ohne Regularisierung 

In [464]:
pipe = Pipeline([("Vorverarbeitung", column_trans), ("linear regression", LinearRegression())])

X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

pipe.fit(X_train_split, y_train_split)

y_train_pred = pipe.predict(X_train_split)
y_test_pred = pipe.predict(X_test_split)

r2_train = r2_score(y_train_split, y_train_pred)
r2_test = r2_score(y_test_split, y_test_pred)

print(f"R^2-Wert auf den Trainingsdaten: {r2_train}")
print(f"R^2-Wert auf den Testdaten {r2_test}")

R^2-Wert auf den Trainingsdaten: 0.602604842138772
R^2-Wert auf den Testdaten 0.6015455496072843


<br>

## Lineares Modell (Ridge Regression)

In [465]:
kreuzvalidierung_mit_hyperparameter(model_type="ridge")

Vergleichstabelle der R^2 Werte:

|   r2_train |   r2_test |   alpha |   validation_dataset |
|-----------:|----------:|--------:|---------------------:|
|   0.602605 |  0.601546 |    0.01 |                    1 |
|   0.602605 |  0.601546 |    0.01 |                    2 |
|   0.602605 |  0.601546 |    0.01 |                    3 |
|   0.602605 |  0.601546 |    0.01 |                    4 |
|   0.602605 |  0.601546 |    0.01 |                    5 |
|   0.602604 |  0.601545 |    0.1  |                    1 |
|   0.602604 |  0.601545 |    0.1  |                    2 |
|   0.602604 |  0.601545 |    0.1  |                    3 |
|   0.602604 |  0.601545 |    0.1  |                    4 |
|   0.602604 |  0.601545 |    0.1  |                    5 |
|   0.602601 |  0.601544 |    0.5  |                    1 |
|   0.602601 |  0.601544 |    0.5  |                    2 |
|   0.602601 |  0.601544 |    0.5  |                    3 |
|   0.602601 |  0.601544 |    0.5  |                    4 |
|   0.

<br>

## Lineares Modell (Lasso Regression)

In [466]:
kreuzvalidierung_mit_hyperparameter(model_type="lasso")

C:\Users\sinan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.992e+11, tolerance: 2.662e+08

C:\Users\sinan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.992e+11, tolerance: 2.662e+08

C:\Users\sinan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:631: Conv

Vergleichstabelle der R^2 Werte:

|   r2_train |   r2_test |   alpha |   validation_dataset |
|-----------:|----------:|--------:|---------------------:|
|   0.602598 |  0.601541 |    0.01 |                    1 |
|   0.602598 |  0.601541 |    0.01 |                    2 |
|   0.602598 |  0.601541 |    0.01 |                    3 |
|   0.602598 |  0.601541 |    0.01 |                    4 |
|   0.602598 |  0.601541 |    0.01 |                    5 |
|   0.602597 |  0.601544 |    0.1  |                    1 |
|   0.602597 |  0.601544 |    0.1  |                    2 |
|   0.602597 |  0.601544 |    0.1  |                    3 |
|   0.602597 |  0.601544 |    0.1  |                    4 |
|   0.602597 |  0.601544 |    0.1  |                    5 |
|   0.602583 |  0.601545 |    0.5  |                    1 |
|   0.602583 |  0.601545 |    0.5  |                    2 |
|   0.602583 |  0.601545 |    0.5  |                    3 |
|   0.602583 |  0.601545 |    0.5  |                    4 |
|   0.

<br>

## Decision Tree Regressor

In [467]:
kreuzvalidierung_mit_hyperparameter(model_type="tree", max_depth_set=[None,5])

Vergleichstabelle der R^2 Werte:

|   r2_train |   r2_test |   max_depth |   min_samples_split |   validation_dataset |
|-----------:|----------:|------------:|--------------------:|---------------------:|
|   1        |  0.923684 |         nan |                   2 |                    1 |
|   1        |  0.923548 |         nan |                   2 |                    2 |
|   1        |  0.923474 |         nan |                   2 |                    3 |
|   1        |  0.923289 |         nan |                   2 |                    4 |
|   1        |  0.923645 |         nan |                   2 |                    5 |
|   0.991282 |  0.930502 |         nan |                   5 |                    1 |
|   0.991282 |  0.930527 |         nan |                   5 |                    2 |
|   0.991282 |  0.930595 |         nan |                   5 |                    3 |
|   0.991282 |  0.930615 |         nan |                   5 |                    4 |
|   0.991282 |  0.93

<br>

## Random Forest Regressor

In [468]:
kreuzvalidierung_mit_hyperparameter(model_type="random_forest", max_depth_set=[10,None])     

Vergleichstabelle der R^2 Werte:

|   r2_train |   r2_test |   max_depth |   min_samples_split |   validation_dataset |
|-----------:|----------:|------------:|--------------------:|---------------------:|
|   0.82244  |  0.817569 |          10 |                   2 |                    1 |
|   0.822057 |  0.817074 |          10 |                   2 |                    2 |
|   0.822955 |  0.818195 |          10 |                   2 |                    3 |
|   0.82355  |  0.818817 |          10 |                   2 |                    4 |
|   0.82255  |  0.817693 |          10 |                   2 |                    5 |
|   0.823754 |  0.819086 |          10 |                   5 |                    1 |
|   0.821982 |  0.817263 |          10 |                   5 |                    2 |
|   0.82242  |  0.81776  |          10 |                   5 |                    3 |
|   0.82285  |  0.8181   |          10 |                   5 |                    4 |
|   0.82203  |  0.81

<br>

# Nr.5

In [469]:
best_alpha = 0.01
best_max_depth = None
best_min_samples_split = [2,5]

pipe_random_forest = Pipeline([("Vorverarbeitung", column_trans), ("random_forest regression", RandomForestRegressor(max_depth=best_max_depth, min_samples_split=best_min_samples_split[0]))])
pipe_ridge = Pipeline([("Vorverarbeitung", column_trans), ("ridge regression", linear_model.Ridge(alpha=best_alpha))])
pipe_tree = Pipeline([("Vorverarbeitung", column_trans), ("tree regression", DecisionTreeRegressor(max_depth=best_max_depth, min_samples_split=best_min_samples_split[1]))])


In [470]:
#Funktion für die Vorhersage des Mitarbeiterbedarfs auf Grundlage der zusätzlich anfallenden / zu viel gezahlten Lohnkosten
def vorhersage_mitarbeiterbedarf(model=best_model, X_train=X_train, y_train=y_train):

    X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


    model.fit(X_train_split,y_train_split)
    y_pred = model.predict(X_test_split)
    
    delta_umsatz = y_pred - y_test_split

    lohnkosten = 0

    # Kosten für Überschätzung des Umsatzes
    lohnkosten += 150 * sum(delta_umsatz > 3000)
    lohnkosten += 250 * sum(delta_umsatz > 6000)

    # Kosten für Unterschätzung des Umsatzes
    lohnkosten += 100 * sum(delta_umsatz < -4000)

    return lohnkosten


### Decision Tree Regressor

In [471]:
print(vorhersage_mitarbeiterbedarf(model= pipe_tree))

79800


### Random Forest Regressor

In [472]:
print(vorhersage_mitarbeiterbedarf(model= pipe_random_forest))

33600


### Ridge Regression

In [473]:
print(vorhersage_mitarbeiterbedarf(model= pipe_ridge))

715500


<br>

# Nr.6

In [474]:
#Vorläufiger Testdatensatz mit dem Stores-Datensatz mergen
df_merge_test_stores = pd.merge(df_stores, df_test, how="outer", on= "Store ID")

#Umwandlung von "Date" in Datumsformat
df_merge_test_stores['Date'] = pd.to_datetime(df_merge_test_stores['Date'])

#Auffüllen der NaN-Werte von "PromoIntervall"
df_merge_test_stores["PromoInterval"] = df_merge_test_stores["PromoInterval"].fillna("NoPromoInterval")

#Generieren neuer Datumsfeatures
df_merge_test_stores['Jahr'] = df_merge_test_stores['Date'].dt.year
df_merge_test_stores['Monat'] = df_merge_test_stores['Date'].dt.month
df_merge_test_stores['Tag'] = df_merge_test_stores['Date'].dt.day

#Feature, das anzeigt, ob Wochenende ist
df_merge_test_stores['IsWeekend'] = df_merge_test_stores['Date'].dt.dayofweek.isin([6, 7]).astype(int)

#Feature, das die Tage bis Weihnachten zählt
df_merge_test_stores['DaysUntilChristmas'] = (pd.to_datetime(df_merge_test_stores['Date'].dt.year.astype(str) + '-12-25') - df_merge_test_stores['Date']).dt.days
df_merge_test_stores.loc[df_merge_test_stores['DaysUntilChristmas'] < 0, 'DaysUntilChristmas'] = (pd.to_datetime((df_merge_test_stores['Date'].dt.year + 1).astype(str) + '-12-25') - df_merge_test_stores['Date']).dt.days

df_merge_test_stores

,Store ID,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Jahr,Monat,Tag,IsWeekend,DaysUntilChristmas
0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",5,2015-07-31,1,1,0,1,2015,7,31,0,147
1,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4,2015-07-30,1,1,0,1,2015,7,30,0,148
2,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",3,2015-07-29,1,1,0,1,2015,7,29,0,149
3,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2,2015-07-28,1,1,0,1,2015,7,28,0,150
4,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",1,2015-07-27,1,1,0,1,2015,7,27,0,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27595,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NoPromoInterval,2,2015-05-05,1,1,0,0,2015,5,5,0,234
27596,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NoPromoInterval,1,2015-05-04,1,1,0,0,2015,5,4,0,235
27597,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NoPromoInterval,7,2015-05-03,0,0,0,0,2015,5,3,1,236
27598,300,a,c,9260.0,NaN,NaN,0,NaN,NaN,NoPromoInterval,6,2015-05-02,1,0,0,0,2015,5,2,0,237


In [475]:
#Testdaten
X_test = df_merge_test_stores.drop(columns=["Date"])

In [476]:
#Modelltraining und -vorhersage
pipe_random_forest.fit(X_train,y_train)
y_pred = pipe_random_forest.predict(X_test)

In [477]:
df_test["Sales"] = y_pred

In [478]:
df_test

,Store ID,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Sales
0,1,5,2015-07-31,1,1,0,1,6071.56
1,1,4,2015-07-30,1,1,0,1,6341.40
2,1,3,2015-07-29,1,1,0,1,6158.42
3,1,2,2015-07-28,1,1,0,1,5891.06
4,1,1,2015-07-27,1,1,0,1,6819.31
...,...,...,...,...,...,...,...,...
27595,300,2,2015-05-05,1,1,0,0,7021.71
27596,300,1,2015-05-04,1,1,0,0,8261.43
27597,300,7,2015-05-03,0,0,0,0,0.00
27598,300,6,2015-05-02,1,0,0,0,7550.81


In [479]:
#Neue csv-Datei für die Testdaten mit der vorhergesagten Spalte "Sales"
df_test.to_csv('dmml1_test_prediction.csv', index=False)
